In [1]:
# the below table defines Ranking and Analytics functions and for aggregate functions, we can
# use any existing aggregate functions as a window function.
#
# To perform an operation on a group first, we need to partition the data using `Window.partitionBy()` 
# and for row number and rank function we need to additionally order by on partition data using 
# `orderBy` clause.
#
# Window Function Syntax       PySpark Window Function Description
#
# row_number(): Column         Returns a sequential number starting from 1 within a window partition
#
# rank(): Column               Returns the rank of rows within a window partition, with gaps
#
# percent_rank(): Column       Returns the percentile rank of rows within a window partition
#
# dense_rank(): Column         Returns the rank of rows within a window partition without any gaps.
#                              Where as rank() rturns rank with gap.
# ntile(n: Int): Column        

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [4]:
simpleData = (("James", "Sales", 3000), \
              ("Michael", "Sales", 4600),  \
              ("Robert", "Sales", 4100),   \
              ("Maria", "Finance", 3000),  \
              ("James", "Sales", 3000),    \
              ("Scott", "Finance", 3300),  \
              ("Jen", "Finance", 3900),    \
              ("Jeff", "Marketing", 3000), \
              ("Kumar", "Marketing", 2000),\
              ("Saif", "Sales", 4100) \
             )

In [5]:
columns= ["employee_name", "department", "salary"]

In [6]:
df = spark.createDataFrame(data = simpleData, schema = columns)

In [7]:
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [8]:
df.show(truncate=False)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [9]:
from pyspark.sql.window import Window

In [10]:
from pyspark.sql.functions import row_number

In [11]:
windowSpec = Window.partitionBy("department").orderBy("salary")

In [12]:
df.withColumn("row_number", row_number().over(windowSpec)) \
                                        .show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



In [13]:
# rank function
from pyspark.sql.functions import rank

In [14]:
df.withColumn("rank",rank().over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
+-------------+----------+------+----+



In [15]:
# dense_rank window function is used to get the result with rank of rows within a window partition without any gaps. 
# this is similar to rank function with difference being rank function leaves gaps in rank when there are ties.
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank", dense_rank().over(windowSpec)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
+-------------+----------+------+----------+



In [16]:
# percent_rank Window Function

from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank", percent_rank().over(windowSpec)).show()

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|        Maria|   Finance|  3000|         0.0|
|        Scott|   Finance|  3300|         0.5|
|          Jen|   Finance|  3900|         1.0|
|        Kumar| Marketing|  2000|         0.0|
|         Jeff| Marketing|  3000|         1.0|
|        James|     Sales|  3000|         0.0|
|        James|     Sales|  3000|         0.0|
|       Robert|     Sales|  4100|         0.5|
|         Saif|     Sales|  4100|         0.5|
|      Michael|     Sales|  4600|         1.0|
+-------------+----------+------+------------+



In [17]:
# ntile window function returns the relative rank of result rows within a window partition.
# In below example we have used 2 as an argument to ntile hence it returns ranking beteen 2 values (1 and 2)

In [18]:
"""ntile"""
from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(2).over(windowSpec)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    1|
|          Jen|   Finance|  3900|    2|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    1|
|         Saif|     Sales|  4100|    2|
|      Michael|     Sales|  4600|    2|
+-------------+----------+------+-----+

